If ever necessary, this part can be done following https://towardsdatascience.com/mapping-with-matplotlib-pandas-geopandas-and-basemap-in-python-d11b57ab5dac

In [2]:
## Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from mlxtend.plotting import plot_decision_regions
import shapefile as shp
import seaborn as sns
from zipfile import ZipFile

In [5]:
## Data Cleaning
# Get energy dummy
sample_data = pd.read_csv('../EXAM/data/cleaned_data.csv',encoding='utf-16')
df = pd.DataFrame(sample_data)
df


,price,address,city,type,saledays,energy,living_space,ground_space,rooms,owner_expenses,...,age_squared,area_digit_1,area_digit_2,area_digit_3,area_digit_4,area_digit_5,area_digit_6,area_digit_7,area_digit_8,area_digit_9
0,1275000,Strandparken 46,Vest- og Sydsjælland,Holiday home,202.0,Energimærke D,170.0,1019.0,2.0,2.598,...,2025.0,0,0,0,1,0,0,0,0,0
1,5095000,"Kalkbrænderihavnsgade 4A, 1. tv.",Byen København,Apartment,0.0,Energimærke A,94.0,0.0,3.0,2.389,...,36.0,0,1,0,0,0,0,0,0,0
2,14750000,Niels Andersens Vej 56,Københavns omegn,Villa,255.0,Energimærke E,248.0,984.0,8.0,8.643,...,6724.0,0,1,0,0,0,0,0,0,0
3,4498000,"Helga Pedersens Gade 1, 2. 3.",Østjylland,Apartment,7.0,Energimærke B,107.0,0.0,3.0,3.849,...,81.0,0,0,0,0,0,0,0,1,0
4,2850000,Nøddevænget 20,Sydjylland,Villa,0.0,Energimærke C,163.0,858.0,5.0,3.343,...,3364.0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,1750000,"Prins Haralds Allé 101, 1.",Fyn,Villa,48.0,Energimærke G,109.0,506.0,3.0,1.922,...,7744.0,0,0,0,0,1,0,0,0,0
3646,990000,Nissumvej 4,Vestjylland,Holiday home,48.0,Energimærke C,117.0,800.0,3.0,1.566,...,6561.0,0,0,0,0,0,0,1,0,0
3647,1695000,Baunetoften 12,Nordsjælland,Terraced house,240.0,Energimærke C,86.0,103.0,4.0,2.184,...,1089.0,0,0,1,0,0,0,0,0,0
3648,650000,Aalevej 40,Østjylland,Villa,48.0,Energimærke D,79.0,829.0,3.0,1.464,...,4624.0,0,0,0,0,0,0,1,0,0


In [6]:
## Dataset split
columns_to_drop = ['price','address','city','type','energy', 'year','area_name','area_digit_1','area_digit_2','area_digit_3','area_digit_4','area_digit_5','area_digit_6','area_digit_7','area_digit_8','area_digit_9']
X,y = df.drop(columns_to_drop,axis=1),df.price
test_size = 0.30 # taking 70:30 training and test set
seed = 17082023 # Random number seeding for repeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
# Standardize X
norm_scaler = StandardScaler().fit(X_train)
X_train_std = norm_scaler.transform(X_train)
X_test_std = norm_scaler.transform(X_test)
X_train_std = pd.DataFrame(X_train_std,columns=X.columns)
X_test_std = pd.DataFrame(X_test_std,columns=X.columns)
X_std = pd.concat([X_train_std,X_test_std],axis=1)

In [7]:
## Initialize visualization set
sns.set(style='whitegrid', palette='pastel', color_codes=True)
sns.mpl.rc('figure', figsize=(10,6))

In [6]:
#opening the vector map
with ZipFile('Denmark_shapefile.zip') as shpfile:
    shpfile.extractall(path='../EXAM/Denmark_shapefile')
shp_path = '../EXAM/Denmark_shapefile/dk_1km.shp'
#reading the shape file by using reader function of the shape lib
sf = shp.Reader(shp_path)
len(sf.shapes())
# sf.records()
sf.records()[1][0]


'1kmE3886N3680'